Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [22]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
from tensorflow.contrib.layers import flatten

In [2]:
pickle_file = 'models/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
    dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [60]:


batch_size = 64
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    keep_prob = tf.placeholder(tf.float32)
  # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    weights = {
        'conv_w1': tf.Variable(tf.random_normal([patch_size, patch_size, num_channels, depth], stddev=0.1)),
        'conv_w2': tf.Variable(tf.random_normal( [patch_size, patch_size, depth, depth], stddev=0.1)),
        'w3': tf.Variable(tf.random_normal( [image_size // 4 * image_size // 4 * depth, num_hidden],
                                           stddev=0.1)),
        'out': tf.Variable(tf.random_normal([num_hidden, num_labels], stddev=0.1))
    }
    
    biases = {
        'conv_b1': tf.Variable(tf.random_normal([depth], stddev=0.1)),
        'conv_b2': tf.Variable(tf.random_normal([depth], stddev=0.1)),
        'b3': tf.Variable(tf.random_normal([num_hidden], stddev=0.1)),
        'out': tf.Variable(tf.random_normal([num_labels], stddev=0.1))
    }

  # Model.
    def model(data):
        
        tf.logging.info("Data: %s", data)
        conv = tf.nn.conv2d(data, weights['conv_w1'], [1, 1, 1, 1], padding='SAME')
        tf.logging.info("Conv1: %s", conv)
        conv = tf.nn.max_pool(conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        tf.logging.info("max_pool1: %s", conv)
        hidden = tf.nn.relu(conv + biases['conv_b1'])
        conv = tf.nn.conv2d(hidden, weights['conv_w2'], [1, 1, 1, 1], padding='SAME')
        tf.logging.info('conv2: %s', conv)
        conv = tf.nn.max_pool(conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        tf.logging.info('max_pool2: %s', conv)
        hidden = tf.nn.relu(conv + biases['conv_b2'])
        shape = hidden.get_shape().as_list()
        tf.logging.info("shape: %s", shape)
        
        
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        tf.logging.info("reshape: %s", reshape)
        tf.logging.info("w3: %s", weights['w3'])
        hidden = tf.nn.dropout(tf.nn.relu(tf.matmul(reshape, weights['w3']) +  biases['b3']), keep_prob)
        out = tf.matmul(hidden,  weights['out']) + biases['out']
        tf.logging.info("out: %s", out)
        return out

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

INFO:tensorflow:Data: Tensor("Placeholder_1:0", shape=(64, 28, 28, 1), dtype=float32)
INFO:tensorflow:Conv1: Tensor("Conv2D:0", shape=(64, 28, 28, 16), dtype=float32)
INFO:tensorflow:max_pool1: Tensor("MaxPool:0", shape=(64, 14, 14, 16), dtype=float32)
INFO:tensorflow:conv2: Tensor("Conv2D_1:0", shape=(64, 14, 14, 16), dtype=float32)
INFO:tensorflow:max_pool2: Tensor("MaxPool_1:0", shape=(64, 7, 7, 16), dtype=float32)
INFO:tensorflow:shape: [64, 7, 7, 16]
INFO:tensorflow:reshape: Tensor("Reshape:0", shape=(64, 784), dtype=float32)
INFO:tensorflow:w3: <tf.Variable 'Variable_2:0' shape=(784, 64) dtype=float32_ref>
INFO:tensorflow:out: Tensor("add_3:0", shape=(64, 10), dtype=float32)
INFO:tensorflow:Data: Tensor("Const:0", shape=(10000, 28, 28, 1), dtype=float32)
INFO:tensorflow:Conv1: Tensor("Conv2D_2:0", shape=(10000, 28, 28, 16), dtype=float32)
INFO:tensorflow:max_pool1: Tensor("MaxPool_2:0", shape=(10000, 14, 14, 16), dtype=float32)
INFO:tensorflow:conv2: Tensor("Conv2D_3:0", shape=(1

In [62]:
num_steps = 5501

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,
                    keep_prob: 0.7}
        
        _, l, predictions = session.run(
        [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 1000 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(feed_dict={keep_prob: 1.0}), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(feed_dict={keep_prob: 1.0}), test_labels))

Initialized
Minibatch loss at step 0: 2.597584
Minibatch accuracy: 9.4%
Validation accuracy: 10.0%
Minibatch loss at step 1000: 0.378585
Minibatch accuracy: 90.6%
Validation accuracy: 85.0%
Minibatch loss at step 2000: 0.486769
Minibatch accuracy: 87.5%
Validation accuracy: 86.3%
Minibatch loss at step 3000: 0.494383
Minibatch accuracy: 89.1%
Validation accuracy: 86.8%
Minibatch loss at step 4000: 0.541781
Minibatch accuracy: 84.4%
Validation accuracy: 87.8%
Minibatch loss at step 5000: 0.500489
Minibatch accuracy: 84.4%
Validation accuracy: 88.1%
Test accuracy: 94.5%


In [13]:
best 95.2%

7.0

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

#### Lenet

In [78]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

batch_size = 64
patch_size = 5
num_filters_1 = 16
fully_connected_1_nodes = 64

graph = tf.Graph()

with graph.as_default():
    keep_prob = tf.placeholder(tf.float32)
  # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    weights = {
        'conv1_w': tf.Variable(tf.random_normal(
            [5, 5, 1, 6], stddev=0.1)),
        'conv2_w': tf.Variable(tf.random_normal(
            [5, 5, 6, 16], stddev=0.1)),
        'fc1_w': tf.Variable(tf.random_normal(
            [256, 120], stddev=0.1)),
        'fc2_w': tf.Variable(tf.random_normal(
            [120, 84],
                                           stddev=0.1)),
        'out': tf.Variable(tf.random_normal([84, 10], stddev=0.1))
    }
    
    biases = {
        'conv1_b': tf.Variable(tf.random_normal([6], stddev=0.1)),
        'conv2_b': tf.Variable(tf.random_normal([16], stddev=0.1)),
        'fc1_b': tf.Variable(tf.random_normal([120], stddev=0.1)),
        'fc2_b': tf.Variable(tf.random_normal([84], stddev=0.1)),
        'out_b': tf.Variable(tf.random_normal([10], stddev=0.1))
    }

  # Model.
    def model(data):
        
        tf.logging.info("Data: %s\n", data)
        #Input 28 * 28 * 1
        #First layer 
        conv1 = tf.nn.conv2d(data, weights['conv1_w'], [1, 1, 1, 1], padding='VALID')
        conv1 = tf.nn.relu(conv1 + biases['conv1_b'])
        tf.logging.info("Conv1: %s\n", conv1) # 24 * 24 * 6
        #Second layer
        avgp_1 = tf.nn.avg_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
        tf.logging.info("avgp_1: %s\n", avgp_1) # 12 * 12 * 6
        #Third layer
        conv2 = tf.nn.conv2d(avgp_1, weights['conv2_w'], [1, 1, 1, 1], padding='VALID')
        conv2 = tf.nn.relu(conv2 + biases['conv2_b'])
        tf.logging.info("conv2: %s\n", conv2) # 8 * 8 * 16
        #Fouth layer
        avgp_2 = tf.nn.avg_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
        tf.logging.info("avgp_2: %s\n", avgp_2) # 4 * 4 * 16
        avgp_2_flattened = flatten(avgp_2)
        tf.logging.info("avgp_2_flattened: %s\n", avgp_2_flattened) # 1 * 1 * 120
        #Fifth layer
        fc1 = tf.nn.dropout(tf.nn.relu(tf.matmul(avgp_2_flattened,  weights['fc1_w']) + biases['fc1_b']), keep_prob)
        tf.logging.info("fc1: %s\n", fc1) # 1 * 120
        #Sixth layer
        fc2 = tf.nn.dropout(tf.nn.relu(tf.matmul(fc1, weights['fc2_w']) + biases['fc2_b']), keep_prob)
        tf.logging.info("fc2: %s\n", fc2) # 1 * 84
        
        #out layer
        out = tf.matmul(fc2, weights['out'])
        tf.logging.info("out: %s\n", out) # 1 * 10

        return out

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits))
    
    # Optimizer.
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.05, global_step, decay_steps =1000, decay_rate=0.99)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)


    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

INFO:tensorflow:Data: Tensor("Placeholder_1:0", shape=(64, 28, 28, 1), dtype=float32)

INFO:tensorflow:Conv1: Tensor("Relu:0", shape=(64, 24, 24, 6), dtype=float32)

INFO:tensorflow:avgp_1: Tensor("AvgPool:0", shape=(64, 12, 12, 6), dtype=float32)

INFO:tensorflow:conv2: Tensor("Relu_1:0", shape=(64, 8, 8, 16), dtype=float32)

INFO:tensorflow:avgp_2: Tensor("AvgPool_1:0", shape=(64, 4, 4, 16), dtype=float32)

INFO:tensorflow:avgp_2_flattened: Tensor("Flatten/flatten/Reshape:0", shape=(64, 256), dtype=float32)

INFO:tensorflow:fc1: Tensor("dropout/mul:0", shape=(64, 120), dtype=float32)

INFO:tensorflow:fc2: Tensor("dropout_1/mul:0", shape=(64, 84), dtype=float32)

INFO:tensorflow:out: Tensor("MatMul_2:0", shape=(64, 10), dtype=float32)

INFO:tensorflow:Data: Tensor("Const:0", shape=(10000, 28, 28, 1), dtype=float32)

INFO:tensorflow:Conv1: Tensor("Relu_4:0", shape=(10000, 24, 24, 6), dtype=float32)

INFO:tensorflow:avgp_1: Tensor("AvgPool_2:0", shape=(10000, 12, 12, 6), dtype=float32)


In [81]:
num_steps = 50501

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,
                    keep_prob: 0.8}
        
        _, l, predictions = session.run(
        [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 1000 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(feed_dict={keep_prob: 1.0}), valid_labels))
            print("Learning Rate: {0}".format(learning_rate.eval()))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(feed_dict={keep_prob: 1.0}), test_labels))

Initialized
Minibatch loss at step 0: 2.384682
Minibatch accuracy: 7.8%
Validation accuracy: 9.1%
Learning Rate: 0.04999949783086777
Minibatch loss at step 1000: 0.488754
Minibatch accuracy: 84.4%
Validation accuracy: 80.4%
Learning Rate: 0.0494995042681694
Minibatch loss at step 2000: 0.568053
Minibatch accuracy: 85.9%
Validation accuracy: 82.8%
Learning Rate: 0.049004506319761276
Minibatch loss at step 3000: 0.521350
Minibatch accuracy: 85.9%
Validation accuracy: 84.0%
Learning Rate: 0.0485144667327404
Minibatch loss at step 4000: 0.693986
Minibatch accuracy: 82.8%
Validation accuracy: 84.6%
Learning Rate: 0.04802931845188141
Minibatch loss at step 5000: 0.574507
Minibatch accuracy: 81.2%
Validation accuracy: 85.8%
Learning Rate: 0.04754902422428131
Minibatch loss at step 6000: 0.242883
Minibatch accuracy: 93.8%
Validation accuracy: 86.3%
Learning Rate: 0.04707353934645653
Minibatch loss at step 7000: 0.510274
Minibatch accuracy: 81.2%
Validation accuracy: 86.9%
Learning Rate: 0.0466

In [ ]:
95.8